In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import statsmodels as sm
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, confusion_matrix, classification_report, recall_score, f1_score, precision_score, plot_roc_curve, RocCurveDisplay, auc, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.pipeline import Pipeline

from pygam import GAM, LinearGAM
from pygam import GAM, s, te
#librería para análisis exploratorio
# import pandas_profiling
from pandas_profiling import ProfileReport

import sys
sys.path.append("..")
import helpers as hp

In [2]:
df = pd.read_csv('compresive_strength_concrete.csv')
df.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
rename_variables = {
    "Cement (component 1)(kg in a m^3 mixture)": "cemento",
    "Blast Furnace Slag (component 2)(kg in a m^3 mixture)" : "blast_furnace_slag",
    "Fly Ash (component 3)(kg in a m^3 mixture)" : "ceniza_volatil",
    "Water  (component 4)(kg in a m^3 mixture)": "agua",
    "Superplasticizer (component 5)(kg in a m^3 mixture)" : "superplasticidad",
    "Coarse Aggregate  (component 6)(kg in a m^3 mixture)" : "agregado_grueso",
    "Fine Aggregate (component 7)(kg in a m^3 mixture)" : "agregado_fino",
    "Age (day)" : "edad",
    "Concrete compressive strength(MPa, megapascals) " : "concrete_compressive_strength",
}
df.rename(columns=rename_variables, inplace=True)
df.drop(columns = ['agua'], inplace=True)

In [4]:
# hp.describe_variables(df)

In [5]:
#ejecutamos pairplot
# sns.pairplot(df)

# profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
# profile.to_notebook_iframe()

In [6]:
X = df.drop(columns = ["concrete_compressive_strength"])
y = df.concrete_compressive_strength

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [7]:
from feature_engine.wrappers import SklearnTransformerWrapper

# pre-processing transformers

# imputar? No, no tenemos muestras nulas a priori

# escalar? Si
sc = SklearnTransformerWrapper(transformer = StandardScaler(), variables = X_train.columns.to_list())
sc.fit(X_train,y_train)

X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

# encoding? No, no hay variables categoricas

In [8]:
# Modelo sin regularización (Probablemente sobreajuste)
gam_sin_reg = LinearGAM().fit(X_train, y_train)
# y_pred_reg = gam_no_reg.predict(X_test)

In [9]:
# Modelo con regularización (Probablemente subajuste)
# gam = LinearGAM(n_splines=25)
# gam = LinearGAM(s(0) + s(1) + s(2)+ s(3) + s(4) + s(5) + s(6) + s(7), fit_intercept=True) # 8 vars
gam = LinearGAM(s(0) + s(1) + s(2)+ s(3) + s(4) + s(5) + s(6), fit_intercept=True)

lam = np.logspace(-3,3,3)
lams = [lam] * len(X_train.columns)

lams
gam.gridsearch(X_train.values, y_train.values, lam=lams, progress=False)
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     87.8666
Link Function:                     IdentityLink Log Likelihood:                                 -3036.4767
Number of Samples:                          721 AIC:                                             6250.6866
                                                AICc:                                            6275.9939
                                                GCV:                                                33.789
                                                Scale:                                             26.4684
                                                Pseudo R-Squared:                                   0.9176
Feature Function                  Lam

C:\Users\tomas\AppData\Local\Temp\ipykernel_18776\1724567365.py:11: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  gam.summary()


In [10]:
gam_scores = hp.report_regression_metrics(gam, X_test, y_test)
gam_sin_reg_scores = hp.report_regression_metrics(gam_sin_reg, X_test, y_test)

models = {'gam':gam_scores,'gam_sin_reg':gam_sin_reg_scores}
hp.reporte_modelos(models)


# def reporte_modelos2(models_dict):
#     # models_dict
#     models = list(models_dict.keys())
#     metrics = models_dict[models[0]].keys()
#     table_str = '<table><tr><th>Models</th><th>' + '</th><th>'.join(metrics) + '</th><th>Nº de variables</th><th>fit time</th><th>score time</th></tr>'
#     for model in models:
#         table_str += '<tr>'
#         table_str += f"<td>{model}</td>"
#         for metric in metrics:
#             table_str += f"<td>{models_dict[model][metric]:.4f}</td>"
#         table_str += '</tr>'
#     display(HTML(table_str))
# reporte_modelos2(models)


Test R^2: 0.857
Test RMSE: 6.228
Test Median Absolute Error:3.865
Test R^2: 0.855
Test RMSE: 6.262
Test Median Absolute Error:4.164


Models,r2_score,rmse,mae
gam,0.8570,6.2280,3.8650
gam_sin_reg,0.8550,6.2620,4.1640


In [11]:
# XX = gam.generate_X_grid(term=0, n=500)

# plt.plot(XX, gam.predict(XX), 'r--')
# XX, 
# plt.plot(XX, gam.prediction_intervals(XX, width=.95), color='b', ls='--')
# YY = gam.prediction_intervals(XX, width=.95)
# plt.fill_between(XX, gam.prediction_intervals(XX, width=.95)[0], gam.prediction_intervals(XX, width=.95)[1],color='b',alpha=.1)

# plt.scatter(X.values, y.values, facecolor='gray', edgecolors='none')
# plt.title('95% prediction interval')


In [12]:
from pygam.datasets import mcycle

X, y = mcycle(return_X_y=True)
y

1       0.0
2      -1.3
3      -2.7
4       0.0
5      -2.7
       ... 
129   -14.7
130    -2.7
131    10.7
132    -2.7
133    10.7
Name: accel, Length: 133, dtype: float64